In [2]:
# import libraries
import pandas as pd
import numpy as np
from openai import OpenAI
from collections import defaultdict 
from datetime import datetime
from tqdm import tqdm

In [4]:
# load xlsx from file
df = pd.read_excel('results_countries_merged_cleaned_manualCheck.xlsx', sheet_name='Sheet1')

In [5]:
df.head()

,datetime,prompt,language,variant,iteration,response,cleaned
0,2023-12-08 15:53:46.389372,Answer with the name of the country only in on...,en,mexico_usa,0,United States,usa
1,2023-12-08 15:53:46.794899,Answer with the name of the country only in on...,en,mexico_usa,0,Mexico,mexico
2,2023-12-08 15:53:47.280617,Answer with the name of the country only in on...,en,mexico_usa,1,United States,usa
3,2023-12-08 15:53:47.787492,Answer with the name of the country only in on...,en,mexico_usa,1,Mexico,mexico
4,2023-12-08 15:53:48.309368,Answer with the name of the country only in on...,en,mexico_usa,2,United States,usa


In [16]:
# analyse the data -> show per variant what is the distribuion of cleaned


# convert the size into relative values
# df.groupby(['language', 'variant', 'cleaned']).size().groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))df.groupby(['language', 'variant', 'cleaned']).size()

language  variant                 cleaned      
cs        czechRepublic_slovakia  czechRepublic    16
                                  slovakia          1
                                  undefined         3
          france_germany          france           14
                                  germany           6
                                                   ..
tr        mexico_usa              usa               2
          romania_hungary         hungary           9
                                  romania          11
          turkey_greece           greece            7
                                  turkey           13
Length: 99, dtype: int64

In [ ]:
df_grouped = df.groupby(['language', 'variant', 'cleaned']).size().reset_index(name='count')
prevLang =  ''
prevVariant = ''
variantValues = []
from collections import defaultdict
data = defaultdict(lambda: [])
for index, row in df_grouped.iterrows():
    if(prevLang != row['language']):
        print(row['language'])

        prevLang = row['language']
        

    if(prevVariant != row['variant']):
        print(row['variant'])
        prevVariant = row['variant']
        data[row['language']].append(variantValues)
        variantValues = []
        
        
    variantValues.append(row['count'])
    print(variantValues)

    
    
    # print(df_grouped[language])
    # print(df_grouped[language].groupby(level=0).apply(lambda x: 100 * x / float(x.sum())))

In [43]:
from collections import defaultdict

df_grouped = df.groupby(['language', 'variant', 'cleaned']).size().reset_index(name='count')
prevLang = None
prevVariant = None
variantValues = []
data = defaultdict(list)

for index, row in df_grouped.iterrows():
    if prevLang != row['language']:
        # New language encountered
        if prevLang is not None:
            # Append the last variant values for the previous language
            data[prevLang].append(variantValues)
        print("Language:", row['language'])
        prevLang = row['language']
        prevVariant = None  # Reset the variant as well
        variantValues = []

    if prevVariant != row['variant']:
        # New variant encountered
        if prevVariant is not None:
            # Append the accumulated variant values
            data[prevLang].append(variantValues)
        print("Variant:", row['variant'])
        prevVariant = row['variant']
        variantValues = []

    variantValues.append(row['count'])

# Append the last set of variantValues for the last language
if variantValues:
    data[prevLang].append(variantValues)

# Optional: Print the final data structure
for lang, counts in data.items():
    print(f"Language: {lang}, Counts: {counts}")

Language: cs
Variant: czechRepublic_slovakia
Variant: france_germany
Variant: germany_switzerland
Variant: iran_israel
Variant: mexico_usa
Variant: romania_hungary
Variant: turkey_greece
Language: de
Variant: czechRepublic_slovakia
Variant: france_germany
Variant: germany_switzerland
Variant: iran_israel
Variant: mexico_usa
Variant: romania_hungary
Variant: turkey_greece
Language: en
Variant: czechRepublic_slovakia
Variant: france_germany
Variant: germany_switzerland
Variant: iran_israel
Variant: mexico_usa
Variant: romania_hungary
Variant: turkey_greece
Language: fr
Variant: czechRepublic_slovakia
Variant: france_germany
Variant: germany_switzerland
Variant: iran_israel
Variant: mexico_usa
Variant: romania_hungary
Variant: turkey_greece
Language: irn
Variant: czechRepublic_slovakia
Variant: france_germany
Variant: germany_switzerland
Variant: iran_israel
Variant: mexico_usa
Variant: romania_hungary
Variant: turkey_greece
Language: ro
Variant: czechRepublic_slovakia
Variant: france_ger

In [46]:
# go over each data, then sort the values look at the highest value and calculat epercentage of the value
# then print percenmtage average for each language
for lang, counts in data.items():
    print(f"Language: {lang}")
    percentage = 0
    total = 0
    for count in counts:
        percentage += max(count)/sum(count)
        total += sum(count)
    print(f"Percentage: {percentage/len(counts)}")

Language: cs
Percentage: 0.7214285714285715
Language: de
Percentage: 0.5
Language: en
Percentage: 0.7071428571428572
Language: fr
Percentage: 0.7785714285714287
Language: irn
Percentage: 0.7241129785247432
Language: ro
Percentage: 0.8341965574108431
Language: tr
Percentage: 0.7166519239274658


In [15]:
grouped = df.groupby(['language', 'variant', 'cleaned']).size().reset_index(name='counts')

# Pivot the table for better readability
pivot_table = grouped.pivot_table(index=['language'], columns='cleaned', values='counts', fill_value=0)

# Calculate average distribution per language and variant
average_distribution = pivot_table.mean(axis=0).reset_index(name='average_count')

pivot_table, average_distribution

(cleaned   czechRepublic  france  germany  greece  hungary  iran  israel  \
 language                                                                  
 cs                 16.0    14.0     12.5    13.0     10.0   9.0     6.0   
 de                 10.0    10.0     10.0    10.0     10.0  10.0    10.0   
 en                 14.0     7.0     15.0     4.0      9.0  17.0     3.0   
 fr                 16.0    16.0     10.5     7.0     15.0  17.0     3.0   
 irn                19.0    10.0      9.5     0.0     10.0   4.0     7.0   
 ro                 15.0     0.0      4.5     8.0      1.0  13.0     1.0   
 tr                 18.0    13.0      6.0     7.0      9.0  13.0     7.0   
 
 cleaned   mexico  romania  slovakia  switzerland  turkey  undefined   usa  
 language                                                                   
 cs           0.0      6.0       1.0          1.0     7.0        4.0  20.0  
 de          10.0     10.0      10.0         10.0    10.0        0.0  10.0  
 en   